In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelBinarizer
from sklearn.datasets import load_iris


In [2]:
data = load_iris()
label_binarizer = LabelBinarizer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = label_binarizer.fit_transform(data.target)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scalar = StandardScaler()
X_train = scalar.fit_transform(X_train)
X_test = scalar.transform(X_test)


In [3]:
# Innitialize Network
def initialize_network(n_inputs, n_hidden, n_outputs):
    np.random.seed(42)
    network = []
    layer_size = [input_size] + hidden_size + [output_size]
    for i in range(len(layer_size) - 1):
        layer = {

                 'weights' : np.random.randn(layer_size[i], layer_size[i+1])*0.1,
                 'bias' : np.zeros((1,layer_size[i+1]))
                }
        network.append(layer)
        return network

In [4]:
input_size = X_train.shape[1]
hidden_size = [8,8]
output_size = y_train.shape[1]
network = initialize_network(input_size, hidden_size, output_size)

In [5]:
# Sigmoid
def sigmoid(X):
  return 1 / (1 + np.exp(-x))

In [6]:
# Sigmoid derrivative
def sigmoid_derivative(x):
  return x * (1 - x)

In [7]:
# Forward propagation
def forward_prop(network, input):
  activation = input
  for layer in network:
    z = np.dot(activation, layer['weights']) + layer['bias']
    activations = sigmoid(z)


  return activations


In [8]:
def backward_propagation(network, inputs, targets, learning_rate):
  forward_prop(network, input)
  for i in reversed(range(len(network))):
    layer = network[i]
    if i == len(network) - 1:
      layer['delta'] = (layer['output'] - targets) * sigmoid_derivative(layer['output'])
    else:
      next_layer = network[i+1]
      layer['delta'] = np.dot(next_layer['delta'], next_layer['weights'].T) * sigmoid_derivative(layer['activations'])
    for i in range(len(network)):
      layer = network[i]
      if i == 0:
        input_to_use = inputs
      else:
        input_to_use = network[i-1]['activations']
      layer['weights'] -= learning_rate * np.dot(input_to_use.T, layer['delta'])
      layer['biases'] -= learning_rate * np.sum(layer['delta'], axis=0, keepdims=True)

In [9]:
# Train the network
def train_network(network, inputs, targets, learning_rate, epochs):
  for epoch in range(epochs):
    backward_propagation(network, inputs, targets, learning_rate)
    if (epoch + 1) % 100 == 0:
      loss = np.mean(np.square(targets - forward_prop(network, inputs)))
      print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss:.4f}")


train_network(network, X_train, y_train, learning_rate=0.1, epochs=1000)



In [ ]:
# Evaluate the network
def predict(network, input):
  output = forward_prop(network, input)
  return np.argmax(output)

In [ ]:
predictions = []
for i in range(len(X_test)):
  prediction = predict(network, X_test[i])
  predictions.append(prediction)